In [ ]:
import pandas as pd
import pyodbc
from ydata_profiling import ProfileReport

In [ ]:
server = 'DAP-SQL01\CDS' 
database = 'Place'

# ENCRYPT defaults to yes starting in ODBC Driver 18. It's good to always specify ENCRYPT=yes on the client side to avoid MITM attacks.
cnxn = pyodbc.connect(driver='{SQL Server Native Client 11.0}', 
                      host=server, database=database, trusted_connection='yes')

In [ ]:
query = '''
    SELECT start_msoa, end_msoa, hour_part, journey_purpose, journey_purpose_direction, journey_mode, avg_daily_trips
    FROM Process.tb_O2MOTION_ODMODE_Weekly
    WHERE start_date = '2023-03-27'
    '''

In [ ]:
mode_trips = pd.read_sql_query(query,cnxn) 

In [ ]:
mode_trips

In [ ]:
query = '''
    SELECT DISTINCT start_msoa, end_msoa
    FROM Process.tb_O2MOTION_ODMODE_Weekly
    WHERE start_date = '2023-03-27'
'''

In [ ]:
unique_msoa_pairs = pd.read_sql_query(query,cnxn) 
n_unique_msoa_pairs = len(unique_msoa_pairs)
total_msoas = mode_trips['start_msoa'].nunique()


In [ ]:
n_unique_msoa_pairs

In [ ]:
msoa_pair_perc_coverage = round(100 * (n_unique_msoa_pairs / total_msoas ** 2), 1)

In [ ]:
msoa_pair_perc_coverage

In [ ]:
prof = ProfileReport(
                    df=mode_trips,
                    variables={
                                "descriptions": {
                                    "start_msoa": "The Middle Super Output Area in which the journey started. Journeys starting in Scotland and Northern Ireland will use Intermediate Zones and Super Output areas respectively",
                                    "end_msoa": "The Middle Super Output Area in which the journey ended. Journeys ending in Scotland and Northern Ireland will use Intermediate Zones and Super Output areas respectively",
                                    "hour_part": "Hour of start time of journey",
                                    "journey_purpose": "Split into two segments: Commute: A commute journey is defined as a journey between a user’s home location and work location. Other: All other journeys",
                                    "journey_purpose_direction": "Split into following segments: Outbound Home Based Work (OB_HBW) A journey from the user’s home location to their work location. Inbound Home Based Work (IB_HBW) A journey into the user’s home location from their home location. Outbound Home Based Other (OB_HBO) A journey from the user’s home location to a location that’s not their work location. Inbound Home Based Other (IB_HBO) A journey to the user’s home location from a location that’s not their work location. Non-home-based work (NHBW) A journey that neither starts or ends in the user’s home location, but starts or ends in their work location. Non-home-based other (NHBO) A journey that neither starts nor ends in the user’s home or work location",
                                    "journey_mode": "Mode of travel split into Rail and Other trips",
                                    "avg_daily_trips": "Average number of daily trips, for MSOA pair, by hour, by journey purpose, and mode. Note that MSOA pairs with 0 trips are not included in the data. There are " + str(n_unique_msoa_pairs) + " unique MSOA pairs in this cut of the data, this represents approximately " + str(msoa_pair_perc_coverage) + "% of the total possible MSOA pairs"}
                             },
                    title="2. O2 mobility data EDA report - Weekday MODE data (see Dataset tab for SQL query)",
                    dataset={
                        "description": "This report was generated using a single week cut of the full dataset. The following query was used to access this cut of the data:  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎  ‎ ‎ ‎ ‎ ‎ ‎  ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ SELECT start_msoa, end_msoa, hour_part, journey_purpose, journey_purpose_direction, journey_mode, avg_daily_trips ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ FROM Process.tb_O2MOTION_ODMODE_Weekly ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ WHERE start_date = '2023-03-27'",
                        "url": "https://mhclg.sharepoint.com/:w:/s/SpatialDataUnit/EdR1AcS_WzNLr0Ilwt2Ow-4BJNHX-lMxGSly_4MXAK1y4g?e=CDkoXe"
                    },
                    html={"style" : {"theme" : "flatly"}},
                    correlations=None,
                    missing_diagrams=None,
                    duplicates=None,
                    interactions=None
                    )
prof.to_file(output_file='mobility eda - 2. weekday mode data.html')
del mode_trips

In [ ]:
#Jan date for comparison
query = '''
    SELECT start_msoa, end_msoa, hour_part, journey_purpose, journey_purpose_direction, journey_mode, avg_daily_trips
    FROM Process.tb_O2MOTION_ODMODE_Weekly
    WHERE start_date = '2023-01-23'
    '''

In [ ]:
# commented out query as no need to re-run
# mode_trips = pd.read_sql_query(query,cnxn) 
# len(mode_trips)

In [ ]:
# prof = ProfileReport(
#                     df=mode_trips,
#                     variables={
#                                 "descriptions": {
#                                     "start_msoa": "The Middle Super Output Area in which the journey started. Journeys starting in Scotland and Northern Ireland will use Intermediate Zones and Super Output areas respectively",
#                                     "end_msoa": "The Middle Super Output Area in which the journey ended. Journeys ending in Scotland and Northern Ireland will use Intermediate Zones and Super Output areas respectively",
#                                     "hour_part": "Hour of start time of journey",
#                                     "journey_purpose": "Split into two segments: Commute: A commute journey is defined as a journey between a user’s home location and work location. Other: All other journeys",
#                                     "journey_purpose_direction": "Split into following segments: Outbound Home Based Work (OB_HBW) A journey from the user’s home location to their work location. Inbound Home Based Work (IB_HBW) A journey into the user’s home location from their home location. Outbound Home Based Other (OB_HBO) A journey from the user’s home location to a location that’s not their work location. Inbound Home Based Other (IB_HBO) A journey to the user’s home location from a location that’s not their work location. Non-home-based work (NHBW) A journey that neither starts or ends in the user’s home location, but starts or ends in their work location. Non-home-based other (NHBO) A journey that neither starts nor ends in the user’s home or work location",
#                                     "journey_mode": "Mode of travel split into Rail and Other trips",
#                                     "avg_daily_trips": "Average number of daily trips, for MSOA pair, by hour, by journey purpose, and mode"}
#                              },
#                     title="1. O2 mobility data EDA report - Weekday MODE data (see Dataset tab for SQL query)",
#                     dataset={
#                         "description": "This report was generated using a single week cut of the full dataset. The following query was used to access this cut of the data:  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎  ‎ ‎ ‎ ‎ ‎ ‎  ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ SELECT start_msoa, end_msoa, hour_part, journey_purpose, journey_purpose_direction, journey_mode, avg_daily_trips ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ FROM Process.tb_O2MOTION_ODMODE_Weekly ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ WHERE start_date = '2023-01-23'",
#                         "url": "https://mhclg.sharepoint.com/:w:/s/SpatialDataUnit/EdR1AcS_WzNLr0Ilwt2Ow-4BJNHX-lMxGSly_4MXAK1y4g?e=CDkoXe"
#                     },
#                     html={"style" : {"theme" : "flatly"}},
#                     correlations=None,
#                     missing_diagrams=None,
#                     duplicates=None,
#                     interactions=None
#                     )
# prof.to_file(output_file='mode_data_eda_report_JAN.html')

In [ ]:
#weekend mode query
query = '''
    SELECT start_msoa, end_msoa, hour_part, journey_purpose, journey_purpose_direction, journey_mode, avg_daily_trips
    FROM Process.tb_O2MOTION_ODMODE_Weekly
    WHERE start_date = '2023-03-18'
    '''

In [ ]:
mode_trips = pd.read_sql_query(query,cnxn) 
len(mode_trips)

In [ ]:
prof = ProfileReport(
                    df=mode_trips,
                    variables={
                                "descriptions": {
                                    "start_msoa": "The Middle Super Output Area in which the journey started. Journeys starting in Scotland and Northern Ireland will use Intermediate Zones and Super Output areas respectively",
                                    "end_msoa": "The Middle Super Output Area in which the journey ended. Journeys ending in Scotland and Northern Ireland will use Intermediate Zones and Super Output areas respectively",
                                    "hour_part": "Hour of start time of journey",
                                    "journey_purpose": "Split into two segments: Commute: A commute journey is defined as a journey between a user’s home location and work location. Other: All other journeys",
                                    "journey_purpose_direction": "Split into following segments: Outbound Home Based Work (OB_HBW) A journey from the user’s home location to their work location. Inbound Home Based Work (IB_HBW) A journey into the user’s home location from their home location. Outbound Home Based Other (OB_HBO) A journey from the user’s home location to a location that’s not their work location. Inbound Home Based Other (IB_HBO) A journey to the user’s home location from a location that’s not their work location. Non-home-based work (NHBW) A journey that neither starts or ends in the user’s home location, but starts or ends in their work location. Non-home-based other (NHBO) A journey that neither starts nor ends in the user’s home or work location",
                                    "journey_mode": "Mode of travel split into Rail and Other trips",
                                    "avg_daily_trips": "Average number of daily trips, for MSOA pair, by hour, by journey purpose, and mode. Note that MSOA pairs with 0 trips are not included in the data. There are " + str(n_unique_msoa_pairs) + " unique MSOA pairs in this cut of the data, this represents approximately " + str(msoa_pair_perc_coverage) + "% of the total possible MSOA pairs"}
                             },
                    title="3. O2 mobility data EDA report - Weekend MODE data (see Dataset tab for SQL query)",
                    dataset={
                        "description": "This report was generated using a two-weekend cut of the full dataset. The following query was used to access this cut of the data:  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎  ‎ ‎ ‎ ‎ ‎ ‎  ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ SELECT start_msoa, end_msoa, hour_part, journey_purpose, journey_purpose_direction, journey_mode, avg_daily_trips ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ FROM Process.tb_O2MOTION_ODMODE_Weekly ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ WHERE start_date = '2023-03-18'",
                        "url": "https://mhclg.sharepoint.com/:w:/s/SpatialDataUnit/EdR1AcS_WzNLr0Ilwt2Ow-4BJNHX-lMxGSly_4MXAK1y4g?e=CDkoXe"
                    },
                    html={"style" : {"theme" : "flatly"}},
                    correlations=None,
                    missing_diagrams=None,
                    duplicates=None,
                    interactions=None
                    )
prof.to_file(output_file='mobility eda - 3. weekend mode data.html')
del mode_trips


In [ ]:
#weekday demographic data
#for some reason I had to explicitly state the Place schema here otherwise the query would fail
demo_query = '''
    SELECT start_msoa, end_msoa, hour_part, age, gender, spend_power, avg_daily_trips
    FROM [Place].[Process].[tb_O2MOTION_ODDEMO_Weekly]
    WHERE start_date = '2023-03-27'
    '''

In [ ]:
demo_trips = pd.read_sql_query(demo_query, cnxn) 

In [ ]:
prof = ProfileReport(
                    df=demo_trips,
                    variables={
                                "descriptions": {
                                    "start_msoa": "The Middle Super Output Area in which the journey started. Journeys starting in Scotland and Northern Ireland will use Intermediate Zones and Super Output areas respectively",
                                    "end_msoa": "The Middle Super Output Area in which the journey ended. Journeys ending in Scotland and Northern Ireland will use Intermediate Zones and Super Output areas respectively",
                                    "hour_part": "Hour of start time of journey",
                                    "age": "Age band of user",
                                    "gender": "Gender category of user. Note that Age & Gender are based on data supplied by customers when they take out a contract product. No Age & Gender data is collected for other products such as Pay As You Go. The Unknown category makes up around 55% of the users used in the analysis",
                                    "spend_power": "Classification of users propensity to spend. Spend power is inferred from behaviour rather than user home location",
                                    "avg_daily_trips": "Average number of daily trips, for MSOA pair, by hour, by journey purpose, and mode. Note that MSOA pairs with 0 trips are not included in the data. There are 4.2 million unique MSOA pairs in this cut of the data, this represents approximately 4.8% of the total possible MSOA pairs"}
                             },
                    title="4. O2 mobility data EDA report - Weekday demographic data (see Dataset tab for SQL query)",
                    dataset={
                        "description": "This report was generated using a single week cut of the full dataset. The following query was used to access this cut of the data:  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎  ‎ ‎ ‎ ‎ ‎ ‎  ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ SELECT start_msoa, end_msoa, hour_part, age, gender, spend_power, avg_daily_trips ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ FROM Process.tb_O2MOTION_ODDEMO_Weekly ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ WHERE start_date = '2023-03-27'",
                        "url": "https://mhclg.sharepoint.com/:w:/s/SpatialDataUnit/EdR1AcS_WzNLr0Ilwt2Ow-4BJNHX-lMxGSly_4MXAK1y4g?e=CDkoXe"
                    },
                    html={"style" : {"theme" : "flatly"}},
                    correlations=None,
                    missing_diagrams=None,
                    duplicates=None,
                    interactions=None
                    )
prof.to_file(output_file='mobility eda - 4. weekday demographic data.html')
del demo_trips

In [ ]:
#timeseries query - weekday trips by journey mode
timeseries_mode_query = '''
    SELECT start_date, journey_mode, SUM(avg_daily_trips) avg_daily_trips
    FROM [Place].[Process].[tb_O2MOTION_ODMODE_Weekly]
    WHERE DATEPART(dw, start_date) = 1
    GROUP BY start_date, journey_mode
    '''

In [ ]:
timeseries_mode = pd.read_sql_query(timeseries_mode_query, cnxn) 

In [ ]:
timeseries_mode_pivoted = timeseries_mode.pivot(index='start_date', columns='journey_mode', values='avg_daily_trips').reset_index()
timeseries_mode_pivoted.rename(columns={"Rail": "rail_mode_trips", "Other": "other_mode_trips"}, inplace=True)
timeseries_mode_pivoted = timeseries_mode_pivoted.rename_axis(None, axis=1)  
timeseries_mode_pivoted


In [ ]:
#timeseries query - weekday trips by journey purpose
timeseries_purpose_query = '''
    SELECT start_date, journey_purpose, SUM(avg_daily_trips) avg_daily_trips
    FROM [Place].[Process].[tb_O2MOTION_ODMODE_Weekly]
    WHERE DATEPART(dw, start_date) = 1
    GROUP BY start_date, journey_purpose
    '''

In [ ]:
timeseries_purpose = pd.read_sql_query(timeseries_purpose_query, cnxn) 

In [ ]:
timeseries_purpose_pivoted = timeseries_purpose.pivot(index='start_date', columns='journey_purpose', values='avg_daily_trips').reset_index()
timeseries_purpose_pivoted.rename(columns={"Commute": "commute_purpose_trips", "Other": "other_purpose_trips"}, inplace=True)
timeseries_purpose_pivoted= timeseries_purpose_pivoted.rename_axis(None, axis=1)  
timeseries_purpose_pivoted

In [ ]:
# join timeseries data
timeseries_combined = pd.merge(left=timeseries_mode_pivoted, right=timeseries_purpose_pivoted, on="start_date")

In [ ]:
#convert to pd datetime type
timeseries_combined["start_date"] = pd.to_datetime(timeseries_combined["start_date"])

In [ ]:
timeseries_combined

In [ ]:
# FYI - The timeseries section of the code won't run on your machine as it stands. I had to do a hacky hard-code fix of the profiling library to get around a known bug: https://github.com/ydataai/ydata-profiling/issues/1433

# Setting what variables are time series
type_schema = {
    "other_mode_trips": "timeseries",
    "rail_mode_trips": "timeseries",
    "commute_purpose_trips": "timeseries",
    "other_purpose_trips": "timeseries"   
}
# create report
prof = ProfileReport(
                    df=timeseries_combined,
                    tsmode=True, 
                    sortby="start_date",
                    type_schema=type_schema,
                    variables={
                                "descriptions": {
                                    "start_date": "Start date of week for 5 day average collection period. The collection period covers Monday to Friday of each week for each start date. The timeseries variables show the daily average trips over the collection period for the specific mode/journey purpose split",
                                    "other_mode_trips": "Average trips by non-rail modes for all journey purposes",
                                    "rail_mode_trips": "Average trips by rail mode for all journey purposes",
                                    "commute_purpose_trips": "Average number of commute journey purpose trips by all modes. A commute journey is defined as a journey between a user’s home location and work location",
                                    "other_purpose_trips": "Average number of non-commute journey purpose trips by all modes"
                                }
                             },
                    title="1. O2 mobility data EDA report - Timeseries data (see Dataset tab for data processing steps)",
                    dataset={
                        "description": "This report was generated using an aggregation of the full dataset. The data processing steps to generate this aggregated data are shown in this Jupyter notebook: https://github.com/communitiesuk/Mobility_SDU/tree/main/notebooks/generate_eda_reports.ipynb",
                        "url": "https://mhclg.sharepoint.com/:w:/s/SpatialDataUnit/EdR1AcS_WzNLr0Ilwt2Ow-4BJNHX-lMxGSly_4MXAK1y4g?e=CDkoXe"
                    },
                    html={"style" : {"theme" : "flatly"}},
                    missing_diagrams=None,
                    duplicates=None
                    )
prof.to_file(output_file='mobility eda - 1. weekday timeseries data.html')